In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 17.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import optuna
import os
import json
import warnings
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# === Load and prepare dataset ===
df = pd.read_csv("cleaned_stock_data_without_arm.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Company', 'Date'])

# Fix commas to save as numeric
df['Price Close'] = df['Price Close'].astype(str).str.replace(",", ".", regex=False)
df['Price Close'] = pd.to_numeric(df['Price Close'], errors='coerce')

# === Lag the Fama-French factors by 1 day ===
for col in ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']:
    df[col] = df.groupby('Company')[col].shift(1)

df = df.dropna(subset=['Price Close', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA'])

# === Create output folders ===
os.makedirs("optuna_results", exist_ok=True)
os.makedirs("optuna_models", exist_ok=True)

# === Sequence generator ===
def create_sequences(features, target, lookback):
    X, y = [], []
    for i in range(lookback, len(features)):
        X.append(features[i - lookback:i])
        y.append(target[i])
    return np.array(X), np.array(y)

# === Optimization function ===
def optimize_lstm(company_data, company_name):
    def objective(trial):
        lookback = trial.suggest_int('lookback', 1, 60, step=5)
        units = trial.suggest_int('units', 32, 256)
        dropout = trial.suggest_float('dropout', 0.1, 0.5)
        batch_size = trial.suggest_categorical('batch_size', [32, 64])
        epochs = trial.suggest_int('epochs', 20, 35)

        features = company_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].values
        target = company_data['Price Close'].values.reshape(-1, 1)

        f_scaler = MinMaxScaler()
        t_scaler = MinMaxScaler()
        features_scaled = f_scaler.fit_transform(features)
        target_scaled = t_scaler.fit_transform(target)

        full_data = np.hstack((features_scaled, target_scaled))
        X, y = create_sequences(full_data, target_scaled, lookback)

        split = int(len(X) * 0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        model = Sequential()
        model.add(LSTM(units, input_shape=(X.shape[1], X.shape[2])))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

        es = EarlyStopping(patience=5, restore_best_weights=True, verbose=0)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        trial.set_user_attr("Company", company_name)
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30, show_progress_bar=True)

    trial_df = study.trials_dataframe()
    trial_df['Company'] = company_name

    return study.best_trial.params, trial_df

# === Select chunk to run ===
chunk_number = 1  # Set this manually: 0, 1, 2, or 3
num_chunks = 4
all_companies = sorted(df['Company'].unique())
chunk_size = len(all_companies) // num_chunks
chunks = [all_companies[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks - 1)]
chunks.append(all_companies[(num_chunks - 1)*chunk_size:])
companies_to_run = chunks[chunk_number]

# === Store all results ===
all_results = []
all_trials_df = []

# === Run optimization per company in this chunk ===
for company in tqdm(companies_to_run, desc=f"Optimizing chunk {chunk_number}"):
    company_data = df[df['Company'] == company].dropna()
    if len(company_data) < 100:
        continue

    print(f"\n🔍 Optimizing for {company}...")
    best_params, trial_df = optimize_lstm(company_data, company)

    result = {"Company": company}
    result.update(best_params)
    all_results.append(result)
    all_trials_df.append(trial_df)

    print(f"✅ Done: {company} — Best Params: {best_params}")

# === Save outputs for this chunk ===
pd.DataFrame(all_results).to_csv(f"optuna_best_params_chunk{chunk_number}.csv", index=False)
pd.concat(all_trials_df, ignore_index=True).to_csv(f"optuna_trials_chunk{chunk_number}.csv", index=False)


Optimizing chunk 1:   0%|          | 0/11 [00:00<?, ?it/s][I 2025-05-01 09:07:48,577] A new study created in memory with name: no-name-3a4d8e2c-496f-4367-b561-9c80c2597768



🔍 Optimizing for CDW.O...


  0%|          | 0/30 [00:00<?, ?it/s]

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step


Optimizing chunk 1:   0%|          | 0/11 [00:31<?, ?it/s]

...

[I 2025-05-01 11:10:57,867] Trial 27 finished with value: 0.033938785877613356 and parameters: {'lookback': 1, 'units': 215, 'dropout': 0.2825104378788469, 'batch_size': 64, 'epochs': 25}. Best is trial 10 with value: 0.03381952308796629.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step


Optimizing chunk 1:  91%|█████████ | 10/11 [2:03:29<10:32, 632.03s/it]

[I 2025-05-01 11:11:18,543] Trial 28 finished with value: 0.05489998203749729 and parameters: {'lookback': 11, 'units': 239, 'dropout': 0.20834446768606857, 'batch_size': 64, 'epochs': 23}. Best is trial 10 with value: 0.03381952308796629.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


Optimizing chunk 1: 100%|██████████| 11/11 [2:03:50<00:00, 675.54s/it]

[I 2025-05-01 11:11:39,444] Trial 29 finished with value: 0.04644304447607962 and parameters: {'lookback': 6, 'units': 191, 'dropout': 0.12791980468506509, 'batch_size': 32, 'epochs': 33}. Best is trial 10 with value: 0.03381952308796629.
✅ Done: INTU.O — Best Params: {'lookback': 1, 'units': 254, 'dropout': 0.10547275773788117, 'batch_size': 64, 'epochs': 23}
